# CGANs - Conditional Generative Adversarial Nets



In [1]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Conv2D, Dense, GlobalAveragePooling2D, LayerNormalization, Add, DepthwiseConv2D, MaxPool2D
)
from utils import PoolingLayer, ResidualBlock, ResidualBlock3x3, ResidualBlock5x5, ResidualBlock7x7, SpatialSE, ChannelSE, ResidualBlockDepthwise3x3, ResidualBlockDepthwise5x5, ResidualBlockDepthwise7x7



# ---------------------------------------------------------
# Softmax Router (no Gumbel). Optional hard mode at inference.
# ---------------------------------------------------------
class SoftmaxRouter(layers.Layer):
    def __init__(self, num_choices, hard_at_inference=False, **kwargs):
        super().__init__(**kwargs)
        self.num_choices = num_choices
        self.hard_at_inference = hard_at_inference
        self.logits_layer = Dense(num_choices)

    def call(self, features, training=None):
        logits = self.logits_layer(GlobalAveragePooling2D()(features))  # (B, K)
        if training or not self.hard_at_inference:
            probs = tf.nn.softmax(logits, axis=-1)                      # (B, K)
        else:
            idx = tf.argmax(logits, axis=-1)
            probs = tf.one_hot(idx, depth=self.num_choices, dtype=tf.float32)
        return probs  # (B, K)



# ------------------------------------------------------------
# Hazard-rate Halting Head (ST hard gate; good gradients)
# ------------------------------------------------------------
class HazardHaltingHead(layers.Layer):
    """
    Predicts a hazard p_t in (0,1) from features; uses a straight-through
    hard decision in forward, with gradients from the sigmoid (Concrete).
    call(...) returns:
      p_soft: [B,1]  (prob of halting at this step, for gradients/metrics)
      h_st  : [B,1]  (hard 0/1 gate with ST gradient)
    """
    def __init__(self, hidden=64, halt_temp=1.0, **kwargs):
        super().__init__(**kwargs)
        self.hidden = int(hidden)
        self.halt_temp = float(halt_temp)

    def build(self, input_shape):
        # input can be [B,H,W,C] or [B,C]; we GAP if rank-4
        if len(input_shape) == 4:
            in_dim = int(input_shape[-1])
        else:
            in_dim = int(input_shape[-1])
        if self.hidden > 0:
            self.mlp = keras.Sequential([
                layers.Dense(self.hidden, activation="swish"),
                layers.Dense(1),
            ])
        else:
            self.mlp = layers.Dense(1) #, bias_initializer=tf.keras.initializers.Constant(-0.2)

    def call(self, feat, *, can_halt_mask, training=None):
        # feat: [B,H,W,C] or [B,C]
        if len(feat.shape) == 4:
            pooled = tf.reduce_mean(feat, axis=[1,2])  # [B,C]
        else:
            pooled = feat                               # [B,C]

        logits = self.mlp(pooled, training=training)    # [B,1]
        p_soft = tf.nn.sigmoid(logits / self.halt_temp) # [B,1]
        p_soft = p_soft * can_halt_mask                 # respect min_steps

        # straight-through hard gate
        h_hard = tf.cast(p_soft > 0.5, p_soft.dtype)    # [B,1]
        h_st   = h_hard + tf.stop_gradient(p_soft - h_hard)
        return p_soft, h_st

# ---------------------------
# Tiny conv stem
# ---------------------------
class ConvStem(layers.Layer):
    def __init__(self, out_ch, **kw):
        super().__init__(**kw)
        self.conv = layers.Conv2D(out_ch, 3, padding="same", use_bias=False)
        self.norm = layers.LayerNormalization()
        self.act  = layers.Activation("swish")

    def call(self, x, training=None):
        x = self.conv(x)
        x = self.norm(x, training=training)
        x = self.act(x)
        return x


# ---------------------------
# Multi-head attention pooling router
# Produces logits over K experts
# ---------------------------
class AttnPoolRouter(layers.Layer):
    def __init__(self, K, heads=2, dim_head=64, mlp_hidden=64, **kw):
        super().__init__(**kw)
        self.K = int(K)
        self.heads = int(heads)
        self.dim_head = int(dim_head)
        self.mlp_hidden = int(mlp_hidden)

        self.q = self.add_weight(
            name="queries", shape=(self.heads, self.dim_head),
            initializer="glorot_uniform", trainable=True)

        self.key_proj = layers.Conv2D(self.heads*self.dim_head, 1, use_bias=False)
        self.val_proj = layers.Conv2D(self.heads*self.dim_head, 1, use_bias=False)

        # Head aggregator -> K logits
        if self.mlp_hidden > 0:
            self.head_mlp = keras.Sequential([
                layers.Dense(self.mlp_hidden, activation="swish"),
                layers.Dense(self.K)
            ])
        else:
            self.head_mlp = layers.Dense(self.K)

    def call(self, x, training=None):
        # x: [B,H,W,C]
        B, H, W, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], tf.shape(x)[3]
        k = self.key_proj(x)  # [B,H,W,heads*dim]
        v = self.val_proj(x)
        k = tf.reshape(k, [B, H*W, self.heads, self.dim_head])  # [B,HW,Hd,D]
        v = tf.reshape(v, [B, H*W, self.heads, self.dim_head])  # [B,HW,Hd,D]
        k = tf.transpose(k, [0,2,1,3])  # [B,heads,HW,dim]
        v = tf.transpose(v, [0,2,1,3])  # [B,heads,HW,dim]

        # queries: [heads, dim] -> [B,heads,1,dim]
        q = tf.expand_dims(self.q, axis=0)
        q = tf.expand_dims(q, axis=2)

        # attn: [B,heads,1,HW]
        attn = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(self.dim_head, x.dtype))
        attn = tf.nn.softmax(attn, axis=-1)

        # pooled heads: [B,heads,1,dim]
        pooled = tf.matmul(attn, v)  # [B,heads,1,dim]
        pooled = tf.squeeze(pooled, axis=2)  # [B,heads,dim]

        # flatten heads
        pooled = tf.reshape(pooled, [B, self.heads*self.dim_head])  # [B, heads*dim]

        logits = self.head_mlp(pooled, training=training)  # [B,K]
        return logits, pooled  # pooled can be used as a feature if needed


class HaltingClassifierHead(layers.Layer):
    """
    Predicts class probabilities; halts when max prob > tau.
    ST gating: forward uses hard threshold; backward uses a smooth sigmoid around tau.
    """
    def __init__(self, num_classes, hidden=64, halt_temp=3.0, tau=0.8, bias_init=-2.5, **kw):
        super().__init__(**kw)
        self.num_classes = int(num_classes)
        self.hidden = int(hidden)
        self.halt_temp = float(halt_temp)
        self.tau = float(tau)

        if hidden > 0:
            self.classifier = keras.Sequential([
                layers.GlobalAveragePooling2D(),
                layers.Dense(hidden, activation="swish"),
                layers.Dense(num_classes)
            ])
        else:
            self.classifier = keras.Sequential([
                layers.GlobalAveragePooling2D(),
                layers.Dense(num_classes)
            ])

        # a tiny scalar bias we add to (max_prob - tau) before the sigmoid
        self.bias = self.add_weight(
            name="halt_bias", shape=(), initializer=tf.keras.initializers.Constant(bias_init),
            trainable=True)

    def call(self, x, training=None):
        logits = self.classifier(x, training=training)          # [B,C]
        probs  = tf.nn.softmax(logits, axis=-1)                 # [B,C]
        maxp   = tf.reduce_max(probs, axis=-1, keepdims=True)   # [B,1]
        tau = tf.cast(self.tau, maxp.dtype)
        halt_temp = tf.cast(self.halt_temp, maxp.dtype)
        z = (maxp - tau) / tf.maximum(tf.constant(1e-6, dtype=maxp.dtype), halt_temp)
        p_soft = tf.nn.sigmoid(z + tf.cast(self.bias, maxp.dtype))                   # [B,1]
        p_hard = tf.cast(maxp > tau, x.dtype)              # [B,1]
        p_st = p_hard + tf.stop_gradient(p_soft - p_hard)
        return probs, p_soft, p_hard, p_st   # class probs, soft gate, hard gate, ST gate


Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  METAL, no compute capability (probably not an Nvidia GPU)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [2]:
# simple CIFAR-10 aug
def cifar_preprocess(x, y):
    x = tf.image.resize_with_crop_or_pad(x, 36, 36)
    x = tf.image.random_crop(x, [tf.shape(x)[0], 32, 32, 3])
    x = tf.image.random_flip_left_right(x)
    return x, y

def make_dataset(x, y, batch=128, train=True):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if train:
        ds = ds.shuffle(5000).batch(batch).map(cifar_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.batch(batch)
    return ds.prefetch(tf.data.AUTOTUNE)


# Usage
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32")/255.0; x_test = x_test.astype("float32")/255.0
y_train = y_train.flatten(); y_test = y_test.flatten()


ds_train = make_dataset(x_train, y_train, batch=128, train=True)
ds_val   = make_dataset(x_test, y_test, batch=256, train=False)


2025-10-14 19:02:54.358155: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-10-14 19:02:54.358187: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-10-14 19:02:54.358194: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-10-14 19:02:54.358225: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-14 19:02:54.358239: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
class AdaptiveRouterBlockTop1Vectorized(layers.Layer):
    """
    Deterministic Top-1 routing (same in train & infer) with ST grads.
    Pooling is disabled: no pooling layers or events are used.
    """
    def __init__(self,
                 branches,
                 min_steps=1,
                 max_steps=5,
                 ponder_lambda=0.0,
                 route_temp=1.0,
                 halt_temp=1.0,
                 name=None):
        super().__init__(name=name)
        assert 1 <= min_steps <= max_steps
        self.branches = branches
        self.K = len(branches)
        self.router = AttnPoolRouter(K=self.K, dim_head=64, mlp_hidden=0)
        self.halt   = HazardHaltingHead(hidden=64, halt_temp=float(halt_temp))
        self._route_temp = float(route_temp)
        self.ponder_lambda = float(ponder_lambda)
        self.min_steps = int(min_steps)
        self.max_steps = int(max_steps)
    @property
    def route_temp(self):
        return self._route_temp
    @route_temp.setter
    def route_temp(self, v: float):
        self._route_temp = float(v)
    @property
    def halt_temp(self):
        return self.halt.halt_temp
    @halt_temp.setter
    def halt_temp(self, v: float):
        self.halt.halt_temp = float(v)
    def get_config(self):
        cfg = super().get_config()
        cfg.update(dict(
            min_steps=self.min_steps,
            max_steps=self.max_steps,
            ponder_lambda=self.ponder_lambda,
            route_temp=self.route_temp,
            halt_temp=self.halt_temp,
            K=self.K,
        ))
        return cfg
    def call(self, features, training=None):
        x = features
        B = tf.shape(x)[0]
        dtype = x.dtype
        halted = tf.zeros([B,1,1,1], dtype=dtype)
        ponder_cost = tf.constant(0.0, dtype=dtype)
        for t in range(self.max_steps):
            router_logits, _ = self.router(x, training=training)              # [B,K]
            probs  = tf.nn.softmax(router_logits / self.route_temp, axis=-1)  # [B,K]
            top_idx = tf.argmax(probs, axis=-1, output_type=tf.int32)         # [B]
            onehot  = tf.one_hot(top_idx, depth=self.K, dtype=dtype)          # [B,K]
            onehot_st = onehot + tf.stop_gradient(probs - onehot)             # ST
            y_list = [br(x, training=training) for br in self.branches]       # K x [B,H,W,C]
            y_stack = tf.stack(y_list, axis=1)                                 # [B,K,H,W,C]
            mask = tf.reshape(onehot_st, [-1, self.K, 1, 1, 1])
            y_sel = tf.reduce_sum(mask * y_stack, axis=1)                      # [B,H,W,C]
            can_halt = tf.cast(t >= self.min_steps - 1, dtype) * tf.ones([B,1], dtype)
            p_soft, h_st = self.halt(y_sel, can_halt_mask=can_halt, training=training)  # [B,1]
            h_st4 = tf.reshape(h_st, [-1,1,1,1])
            halted = tf.clip_by_value(halted + (1.0 - halted) * h_st4, 0.0, 1.0)
            if training and self.ponder_lambda > 0.0:
                running_frac = tf.reduce_mean(1.0 - tf.squeeze(halted, [1,2,3]))
                ponder_cost += tf.cast(running_frac, dtype)
            if not training and tf.executing_eagerly():
                if bool(tf.reduce_all(tf.squeeze(halted, [1,2,3]) > 0.5).numpy()):
                    break
        if training and self.ponder_lambda > 0.0:
            self.add_loss(self.ponder_lambda * ponder_cost)
        return x


In [ ]:
class TempScheduler(keras.callbacks.Callback):
    """
    Linearly (or cosine) anneal route_temp and halt_temp over epochs.
    route: 1.5 -> 0.7
    halt:  3.0 -> 1.5
    """
    def __init__(self, layer_name="adaptive_router",
                 route_start=1.5, route_end=0.7,
                 halt_start=3.0,  halt_end=0.5,
                 epochs=150, mode="cosine"):
        super().__init__()
        self.layer_name = layer_name
        self.rs, self.re = float(route_start), float(route_end)
        self.hs, self.he = float(halt_start),  float(halt_end)
        self.E = int(epochs)
        self.mode = mode

    def _interp(self, e):
        p = min(1.0, e / max(1, self.E-1))
        if self.mode == "cosine":
            p = 0.5*(1 - np.cos(np.pi*p))
        return p

    def on_epoch_end(self, epoch, logs=None):
        p = self._interp(epoch)
        rtemp = self.rs + (self.re - self.rs)*p
        htemp = self.hs + (self.he - self.hs)*p
        layer = self.model.get_layer(self.layer_name)
        layer.route_temp = rtemp
        layer.halt_temp  = htemp
        print(f" > [TempScheduler] epoch {epoch+1}: route_temp={rtemp:.3f}, halt_temp={htemp:.3f}")


class RouterStatsCallback(keras.callbacks.Callback):
    def __init__(self, x_val, y_val, layer_name="adaptive_router", batch_size=256):
        super().__init__()
        self.xv = x_val
        self.yv = y_val
        self.layer_name = layer_name
        self.bs = batch_size

    def on_epoch_end(self, epoch, logs=None):
        layer = self.model.get_layer(self.layer_name)
        T, K = layer.max_steps, layer.K
        steps_hist = np.zeros(T+1, np.int64)
        expert_hist = np.zeros((T, K), np.int64)
        n = len(self.xv)
        for i in range(0, n, self.bs):
            xb = self.xv[i:i+self.bs]
            tb = trace_batch(self.model, xb, layer_name=self.layer_name, force_full=False)
            t_used = tb["top_indices"].shape[0]
            running = tb["running_after"]  # [t_used, B]
            stopped = ~running
            ever = stopped.any(axis=0)
            first = np.argmax(stopped, axis=0)
            used = np.where(ever, first+1, t_used)
            for s in used:
                steps_hist[min(int(s), T)] += 1
            for t in range(t_used):
                ch = tb["top_indices"][t]
                cnt = np.bincount(ch, minlength=K)
                expert_hist[t] += cnt
        avg_steps = np.sum(np.arange(T+1)*steps_hist)/max(1, steps_hist.sum())
        print(f"> [RouterStats] epoch {epoch+1}: avg_steps={avg_steps:.2f}  steps_hist={steps_hist.tolist()} expert_hist={expert_hist.tolist()}")


def trace_and_predict(
    model,
    x_input,
    y_true=None,
    layer_name="adaptive_router",
    force_full=False,   # set True to always loop max_steps, no early exit
):
    layer = model.get_layer(layer_name)
    info = {
        "min_steps": layer.min_steps,
        "max_steps": layer.max_steps,
        "K": layer.K,
        "route_temp": getattr(layer, "route_temp", None),
        "halt_temp": getattr(getattr(layer, "halt", None), "halt_temp", None),
        "unique_pools": bool(getattr(layer, "_pools", []) not in (None, [])),
        "num_pools": len(getattr(layer, "_pools", []) or []),
    }
    pre = keras.Model(model.input, layer.input)
    x_in = tf.convert_to_tensor(x_input)
    x = pre(x_in, training=False)
    B = int(x.shape[0])
    K, T = layer.K, layer.max_steps
    dtype = x.dtype
    halted = np.zeros((B,1,1,1), dtype=np.float32)
    top_indices, probs_list = [], []
    halt_soft_list, halt_hard_list = [], []
    running_after_list = []
    for t in range(T):
        router_logits, _ = layer.router(x, training=False)
        probs = tf.nn.softmax(router_logits / layer.route_temp, axis=-1)
        top_idx = tf.argmax(probs, axis=-1, output_type=tf.int32)
        onehot = tf.one_hot(top_idx, depth=K, dtype=dtype)
        onehot_st = onehot + tf.stop_gradient(probs - onehot)
        y_list = [br(x, training=False) for br in layer.branches]
        y_stack = tf.stack(y_list, axis=1)
        mask = tf.reshape(onehot_st, [-1, K, 1, 1, 1])
        y_sel = tf.reduce_sum(mask * y_stack, axis=1)
        can_halt = float(t >= layer.min_steps - 1)
        can_mask = tf.ones([B,1], dtype) * can_halt
        p_soft, h_st = layer.halt(y_sel, can_halt_mask=can_mask, training=False)
        h_st_np = h_st.numpy()
        halted = np.clip(halted + (1.0 - halted) * h_st_np, 0.0, 1.0)
        running = np.squeeze(1.0 - halted, axis=(1,2,3)) > 0.5
        x = y_sel  # no pooling
        top_indices.append(top_idx.numpy())
        probs_list.append(probs.numpy())
        halt_soft_list.append(np.squeeze(p_soft.numpy(), axis=1).copy())
        halt_hard_list.append(np.squeeze(h_st_np, axis=1).copy())
        running_after_list.append(running.copy())
        if (not force_full) and (not running.any()):
            break
    pred_probs = model(x_in, training=False).numpy()
    pred_label = pred_probs.argmax(axis=-1).astype(np.int32)
    if y_true is not None:
        y_true_arr = np.asarray(y_true).reshape(-1)
        correct = (pred_label == y_true_arr)
    else:
        correct = None
    trace = {
        "top_indices":       np.array(top_indices),
        "probs":             np.array(probs_list),
        "halt_soft":         np.array(halt_soft_list),
        "halt_hard":         np.array(halt_hard_list),
        "running_after":     np.array(running_after_list),
    }
    return {
        "trace": trace,
        "pred_probs": pred_probs,
        "pred_label": pred_label,
        "true_label": None if y_true is None else np.asarray(y_true),
        "layer_info": info,
        "correct": correct,
    }


def trace_batch(model, x_batch, layer_name="adaptive_router_top1", force_full=False):
    """
    Trace routing/halting for a batch. Pooling is disabled.
    Returns a dict with arrays shaped [t_used, B, ...]:
      - top_indices:  chosen expert index per step
      - probs:        soft routing probabilities per step
      - halt_soft:    hazard p_t per step
      - halt_hard:    0/1 applied halts per step (float)
      - running_after: boolean mask after each step (True = still running)
    """
    layer = model.get_layer(layer_name)
    pre = keras.Model(model.input, layer.input)

    x_in = tf.convert_to_tensor(x_batch)
    x = pre(x_in, training=False)

    B = int(x.shape[0])
    K, T = layer.K, layer.max_steps
    running = np.ones((B,), dtype=bool)

    top_indices, probs_list = [], []
    halt_soft_list, halt_hard_list, running_after_list = [], [], []

    for t in range(T):
        # ---- routing (deterministic Top-1) ----
        router_logits, _ = layer.router(x, training=False)                         # [B,K]
        probs = tf.nn.softmax(router_logits / layer.route_temp, axis=-1)           # [B,K]
        top_idx = tf.argmax(probs, axis=-1, output_type=tf.int32)                  # [B]
        onehot  = tf.one_hot(top_idx, depth=K, dtype=x.dtype)
        onehot_st = onehot + tf.stop_gradient(probs - onehot)                      # ST

        # ---- experts (vectorized) + select ----
        y_stack = tf.stack([br(x, training=False) for br in layer.branches], axis=1)  # [B,K,H,W,C]
        y_sel = tf.reduce_sum(tf.reshape(onehot_st, [-1, K, 1, 1, 1]) * y_stack, axis=1)

        # ---- halting (hazard) ----
        can_halt = (t >= layer.min_steps - 1)
        can_mask = tf.ones([B,1], x.dtype) if can_halt else tf.zeros([B,1], x.dtype)
        p_soft, _ = layer.halt(y_sel, can_halt_mask=can_mask, training=False)      # [B,1]
        p_np = tf.squeeze(p_soft, axis=1).numpy()                                   # [B]
        halt_this = (p_np > 0.5) & running & can_halt                               # [B] bool

        running = running & (~halt_this)

        x = y_sel  # no pooling

        # ---- collect step data ----
        top_indices.append(top_idx.numpy())
        probs_list.append(probs.numpy())
        halt_soft_list.append(p_np.copy())
        halt_hard_list.append(halt_this.astype(np.float32))
        running_after_list.append(running.copy())

        # early exit only if no sample is still running
        if (not force_full) and (not running.any()):
            break

    trace = {
        "top_indices":   np.array(top_indices),                 # [t_used, B]
        "probs":         np.array(probs_list),                  # [t_used, B, K]
        "halt_soft":     np.array(halt_soft_list),              # [t_used, B]
        "halt_hard":     np.array(halt_hard_list),              # [t_used, B]
        "running_after": np.array(running_after_list, dtype=bool),  # [t_used, B]
    }
    return trace

def steps_used_from_running(running_after):
    """
    running_after: [t_used, B] bool (True = still running after that step)
    Returns: [B] int steps used (first time running becomes False; else t_used)
    """
    t_used, B = running_after.shape
    # A sample stops running the step it halts; so steps_used is the first index
    # where running becomes False, +1. If never False, it's t_used.
    stopped = ~running_after
    ever_stopped = stopped.any(axis=0)
    first_stop = np.argmax(stopped, axis=0)  # undefined when never stopped, fine below
    return np.where(ever_stopped, first_stop + 1, t_used)


def evaluate_with_router_stats(model, x, y, layer_name="adaptive_router_top1",
                               batch_size=256, force_full=False):
    # accuracy
    loss, acc = model.evaluate(x, y, batch_size=batch_size, verbose=0)

    # router stats
    layer = model.get_layer(layer_name)
    K, T = layer.K, layer.max_steps

    steps_hist = np.zeros(T+1, dtype=np.int64)   # index t for steps=t, last bin for "T or more"
    expert_hist = np.zeros((T, K), dtype=np.int64)
    halt_rate   = np.zeros(T, dtype=np.float64)
    n_seen = 0

    for i in range(0, len(x), batch_size):
        xb = x[i:i+batch_size]
        tb = trace_batch(model, xb, layer_name=layer_name, force_full=force_full)

        t_used, B = tb["top_indices"].shape[0], tb["top_indices"].shape[1]
        n_seen += B

        # expert usage per step (only for steps that exist in this batch)
        for t in range(t_used):
            choices = tb["top_indices"][t]  # [B]
            counts = np.bincount(choices, minlength=K)
            expert_hist[t, :] += counts
            halt_rate[t] += tb["halt_hard"][t].mean()

        # steps used per sample
        steps_used = steps_used_from_running(tb["running_after"])  # [B]
        # cap into histogram (if force_full=False, some batches may stop early)
        for s in steps_used:
            s_idx = min(int(s), T)  # put "==T" also into T bin
            steps_hist[s_idx] += 1

    # normalize
    halt_rate[:t_used] = halt_rate[:t_used] / max(1, (len(x) + batch_size - 1) // batch_size)

    return {
        "loss": float(loss),
        "acc": float(acc),
        "steps_hist": steps_hist,    # length T+1
        "expert_hist": expert_hist,  # [T, K]
        "halt_rate": halt_rate,      # [T] avg hard halts at step t
        "seen": n_seen,
        "K": K,
        "T": T,
    }



def build_adaptive_model_sparse(input_shape=(32,32,3), num_classes=10, filters=32,
                                min_steps=1, max_steps=5, ponder_lambda=0.0,
                                route_temp=1.0, halt_temp=1.0, pool_every_n=1):
    inputs = keras.Input(shape=input_shape)
    # For CNNs on GPU, BatchNorm is faster than LayerNorm:
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)
    x = ResidualBlock(filters=filters)(x)
    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)

    branches = [
        ResidualBlock3x3(filters),
        ResidualBlock5x5(filters),
        ResidualBlockDepthwise5x5(filters),
        ResidualBlockDepthwise7x7(filters),
        ChannelSE(filters)
    ]
    pools = [PoolingLayer(filters=filters, frac_ratio=2.0) for _ in range(max_steps//pool_every_n)]
    # Prefer simple/fast pooling (or a stride-2 Conv2D inside PoolingLayer)
    #pooling  = PoolingLayer(filters=filters, frac_ratio=2.0)

    x = AdaptiveRouterBlockTop1Vectorized(
        branches=branches,
        min_steps=min_steps,
        max_steps=max_steps,
        ponder_lambda=ponder_lambda,
        route_temp=route_temp,
        halt_temp=halt_temp,
        name="adaptive_router",
    )(x)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)  # keep fp32 logits
    return keras.Model(inputs, outputs)

In [5]:
class AdaptiveRouterBlockTop1Vectorized(layers.Layer):
    """
    Deterministic Top-1 routing (same in train & infer) with ST grads.
    Pooling is disabled: no pooling layers or events are used.
    """
    def __init__(self,
                 branches,
                 min_steps=1,
                 max_steps=5,
                 ponder_lambda=0.0,
                 route_temp=1.0,
                 halt_temp=1.0,
                 name=None):
        super().__init__(name=name)
        assert 1 <= min_steps <= max_steps
        self.branches = branches
        self.K = len(branches)
        self.router = AttnPoolRouter(K=self.K, dim_head=64, mlp_hidden=0)
        self.halt   = HazardHaltingHead(hidden=64, halt_temp=float(halt_temp))
        self._route_temp = float(route_temp)
        self.ponder_lambda = float(ponder_lambda)
        self.min_steps = int(min_steps)
        self.max_steps = int(max_steps)
    @property
    def route_temp(self):
        return self._route_temp
    @route_temp.setter
    def route_temp(self, v: float):
        self._route_temp = float(v)
    @property
    def halt_temp(self):
        return self.halt.halt_temp
    @halt_temp.setter
    def halt_temp(self, v: float):
        self.halt.halt_temp = float(v)
    def get_config(self):
        cfg = super().get_config()
        cfg.update(dict(
            min_steps=self.min_steps,
            max_steps=self.max_steps,
            ponder_lambda=self.ponder_lambda,
            route_temp=self.route_temp,
            halt_temp=self.halt_temp,
            K=self.K,
        ))
        return cfg
    def call(self, features, training=None):
        x = features
        B = tf.shape(x)[0]
        dtype = x.dtype
        halted = tf.zeros([B,1,1,1], dtype=dtype)
        ponder_cost = tf.constant(0.0, dtype=dtype)
        for t in range(self.max_steps):
            router_logits, _ = self.router(x, training=training)              # [B,K]
            probs  = tf.nn.softmax(router_logits / self.route_temp, axis=-1)  # [B,K]
            top_idx = tf.argmax(probs, axis=-1, output_type=tf.int32)         # [B]
            onehot  = tf.one_hot(top_idx, depth=self.K, dtype=dtype)          # [B,K]
            onehot_st = onehot + tf.stop_gradient(probs - onehot)             # ST
            y_list = [br(x, training=training) for br in self.branches]       # K x [B,H,W,C]
            y_stack = tf.stack(y_list, axis=1)                                 # [B,K,H,W,C]
            mask = tf.reshape(onehot_st, [-1, self.K, 1, 1, 1])
            y_sel = tf.reduce_sum(mask * y_stack, axis=1)                      # [B,H,W,C]
            can_halt = tf.cast(t >= self.min_steps - 1, dtype) * tf.ones([B,1], dtype)
            p_soft, h_st = self.halt(y_sel, can_halt_mask=can_halt, training=training)  # [B,1]
            h_st4 = tf.reshape(h_st, [-1,1,1,1])
            halted = tf.clip_by_value(halted + (1.0 - halted) * h_st4, 0.0, 1.0)
            if training and self.ponder_lambda > 0.0:
                running_frac = tf.reduce_mean(1.0 - tf.squeeze(halted, [1,2,3]))
                ponder_cost += tf.cast(running_frac, dtype)
            if not training and tf.executing_eagerly():
                if bool(tf.reduce_all(tf.squeeze(halted, [1,2,3]) > 0.5).numpy()):
                    break
        if training and self.ponder_lambda > 0.0:
            self.add_loss(self.ponder_lambda * ponder_cost)
        return x


In [ ]:
FILTERS = 64
MIN_STEPS = 2
MAX_STEPS = 6
PONDER_LAMBDA = 5e-5  # >0 encourages fewer steps
HALT_TEMP = 2.0
ROUTE_TEMP = 2.0
EPOCHS = 50

model = build_adaptive_model_sparse(
    input_shape=(32,32,3),
    num_classes=10,
    filters=FILTERS,
    min_steps=MIN_STEPS,
    max_steps=MAX_STEPS,
    ponder_lambda=PONDER_LAMBDA,         # >0 encourages fewer steps; tune as needed
    halt_temp=HALT_TEMP,
    route_temp=ROUTE_TEMP
)

callbacks = [
    TempScheduler(layer_name="adaptive_router", epochs=EPOCHS, mode="cosine", route_start=ROUTE_TEMP, route_end=0.7, halt_start=HALT_TEMP, halt_end=0.5),
    RouterStatsCallback(x_test, y_test, layer_name="adaptive_router")
]

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()


Model: "model_682"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_2 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 pooling_layer_14 (PoolingL  (None, 16, 16, 64)        2240      
 ayer)                                                           
                                                                 
 residual_block_2 (Residual  (None, 16, 16, 64)        8320      
 Block)                                                          
                                                         

In [17]:
model.fit(ds_train, epochs=50, validation_data=ds_val, callbacks=callbacks)


 > [TempScheduler] epoch 1: route_temp=1.500, halt_temp=1.500
Epoch 1/50


2025-10-14 19:23:57.643041: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.7784 - accuracy: 0.3478

2025-10-14 19:24:40.105106: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


> [RouterStats] epoch 1: avg_steps=2.57  steps_hist=[0, 0, 8002, 105, 48, 1845] expert_hist=[[134, 4379, 35, 5451, 1], [60, 5302, 0, 4638, 0], [81, 6012, 0, 3907, 0], [387, 6648, 0, 2965, 0], [1457, 7116, 0, 1427, 0]]
391/391 [==============================] - 72s 171ms/step - loss: 1.7784 - accuracy: 0.3478 - val_loss: 1.7858 - val_accuracy: 0.3170
 > [TempScheduler] epoch 2: route_temp=1.499, halt_temp=1.499
Epoch 2/50
 > [TempScheduler] epoch 2: route_temp=1.499, halt_temp=1.499
Epoch 2/50
391/391 [==============================] - 63s 160ms/step - loss: 1.5026 - accuracy: 0.4595 - val_loss: 1.4338 - val_accuracy: 0.4788
> [RouterStats] epoch 2: avg_steps=3.42  steps_hist=[0, 0, 5027, 247, 209, 4517] expert_hist=[[641, 6806, 0, 2449, 104], [316, 7723, 0, 1961, 0], [273, 8150, 0, 1577, 0], [406, 8330, 0, 1264, 0], [719, 8339, 0, 942, 0]]
391/391 [==============================] - 63s 160ms/step - loss: 1.5026 - accuracy: 0.4595 - val_loss: 1.4338 - val_accuracy: 0.4788
 > [TempSchedu

KeyboardInterrupt: 

In [18]:
stats = evaluate_with_router_stats(model, x_test, y_test,
                                   layer_name="adaptive_router",
                                   batch_size=512,
                                   force_full=False)

print(f"Test acc: {stats['acc']*100:.2f}%  |  samples: {stats['seen']}")
print("Steps histogram (0..T; last bin = T):", stats["steps_hist"])
print("Halt rate per step:", np.round(stats["halt_rate"][:np.nonzero(stats['expert_hist'].sum(axis=1) > 0)[0][-1]+1], 3))
print("Expert usage at step 1:", stats["expert_hist"][0])
print("Expert usage at step 2:", stats["expert_hist"][1])
print("Expert usage at step 3:", stats["expert_hist"][2])
print("Expert usage at step 4:", stats["expert_hist"][3])
#print("Expert usage at step 5:", stats["expert_hist"][4])
#print("Expert usage at step 6:", stats["expert_hist"][5])


Test acc: 49.42%  |  samples: 10000
Steps histogram (0..T; last bin = T): [   0    0 3616  241  321 5822]
Halt rate per step: [0.    0.363 0.024 0.032 0.043]
Expert usage at step 1: [ 543 7741    2 1666   48]
Expert usage at step 2: [ 386 8257    0 1354    3]
Expert usage at step 3: [ 574 8476    0  949    1]
Expert usage at step 4: [1112 8303    0  584    1]


In [19]:
for i in range(100, 120, 1):
    res = trace_and_predict(model, x_test[i:i+1], y_true=y_test[i:i+1], layer_name="adaptive_router")

    trace = res["trace"]
    print(f"========= Test sample {i} =========")
    print("experts per step:", trace["top_indices"][:, 0])
    print("halt probs:", trace["halt_soft"][:, 0])
    print("pred label:", res["pred_label"][0], "true label:", int(res["true_label"][0]))



========= Test sample 100 =========
experts per step: [1 1]
halt probs: [0.     0.5254]
pred label: 4 true label: 4
========= Test sample 101 =========
experts per step: [1 1]
halt probs: [0.    0.521]
pred label: 4 true label: 5
========= Test sample 101 =========
experts per step: [1 1]
halt probs: [0.    0.521]
pred label: 4 true label: 5
========= Test sample 102 =========
experts per step: [3 3]
halt probs: [0.     0.5366]
pred label: 3 true label: 6
========= Test sample 102 =========
experts per step: [3 3]
halt probs: [0.     0.5366]
pred label: 3 true label: 6
========= Test sample 103 =========
experts per step: [1 1]
halt probs: [0.    0.505]
pred label: 3 true label: 3
========= Test sample 103 =========
experts per step: [1 1]
halt probs: [0.    0.505]
pred label: 3 true label: 3
========= Test sample 104 =========
experts per step: [1 1 1]
halt probs: [0.     0.4595 0.459 ]
pred label: 9 true label: 1
========= Test sample 104 =========
experts per step: [1 1 1]
halt prob

In [ ]:
def build_base_model_4_blocks(input_shape=(32,32,3), num_classes=10, filters=32):
    inputs = keras.Input(shape=input_shape)
    # For CNNs on GPU, BatchNorm is faster than LayerNorm:
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    x = ResidualBlock(filters)(x)
    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)
    x = ResidualBlock(filters)(x)
    x = ResidualBlock(filters)(x)
    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)
    x = ResidualBlock(filters)(x)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)  # keep fp32 logits
    return keras.Model(inputs, outputs)


def build_base_model_6_blocks(input_shape=(32,32,3), num_classes=10, filters=32):
    inputs = keras.Input(shape=input_shape)
    # For CNNs on GPU, BatchNorm is faster than LayerNorm:
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    x = ResidualBlock(filters)(x)
    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)
    x = ResidualBlock(filters)(x)
    x = ResidualBlock(filters)(x)
    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)
    x = ResidualBlock(filters)(x)
    x = ResidualBlock(filters)(x)
    x = PoolingLayer(filters=filters, frac_ratio=2.0)(x)
    x = ResidualBlock(filters)(x)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)  # keep fp32 logits
    return keras.Model(inputs, outputs)


TypeError: __init__() got an unexpected keyword argument 'pool_every_n'

In [9]:
model = build_base_model_4_blocks(
    input_shape=(32,32,3),
    num_classes=10,
    filters=FILTERS
)

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20, batch_size=512
)

Model: "model_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 residual_block_4 (Residual  (None, 32, 32, 32)        20736     
 Block)                                                          
                                                                 
 pooling_layer_1 (PoolingLa  (None, 16, 16, 32)        1120      
 yer)                                                            
                                                          

2025-10-13 16:17:22.540030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - ETA: 0s - loss: 1.8282 - accuracy: 0.3055

2025-10-13 16:17:42.444025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 23s 189ms/step - loss: 1.8282 - accuracy: 0.3055 - val_loss: 2.3907 - val_accuracy: 0.1831
Epoch 2/20
98/98 [==============================] - 15s 151ms/step - loss: 1.4866 - accuracy: 0.4441 - val_loss: 1.9441 - val_accuracy: 0.3035
Epoch 3/20
98/98 [==============================] - 15s 148ms/step - loss: 1.2755 - accuracy: 0.5356 - val_loss: 1.9825 - val_accuracy: 0.3222
Epoch 4/20
98/98 [==============================] - 14s 143ms/step - loss: 1.1392 - accuracy: 0.5867 - val_loss: 1.7945 - val_accuracy: 0.3760
Epoch 5/20
98/98 [==============================] - 14s 144ms/step - loss: 1.0311 - accuracy: 0.6264 - val_loss: 1.2198 - val_accuracy: 0.5541
Epoch 6/20
98/98 [==============================] - 14s 144ms/step - loss: 0.9601 - accuracy: 0.6540 - val_loss: 1.0745 - val_accuracy: 0.6036
Epoch 7/20
98/98 [==============================] - 15s 148ms/step - loss: 0.9133 - accuracy: 0.6695 - val_loss: 1.0224 - val_accuracy: 0.6342
Epoch 8/20

In [10]:
model = build_base_model_6_blocks(
    input_shape=(32,32,3),
    num_classes=10,
    filters=FILTERS
)

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20, batch_size=512
)

Model: "model_62"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_2 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 residual_block_8 (Residual  (None, 32, 32, 32)        20736     
 Block)                                                          
                                                                 
 pooling_layer_3 (PoolingLa  (None, 16, 16, 32)        1120      
 yer)                                                            
                                                          

2025-10-13 16:22:18.933112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - ETA: 0s - loss: 1.7270 - accuracy: 0.3563

2025-10-13 16:22:38.732527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 23s 194ms/step - loss: 1.7270 - accuracy: 0.3563 - val_loss: 2.5958 - val_accuracy: 0.1778
Epoch 2/20
98/98 [==============================] - 16s 158ms/step - loss: 1.3537 - accuracy: 0.5037 - val_loss: 1.6845 - val_accuracy: 0.3908
Epoch 3/20
98/98 [==============================] - 16s 163ms/step - loss: 1.1777 - accuracy: 0.5730 - val_loss: 1.4184 - val_accuracy: 0.4790
Epoch 4/20
98/98 [==============================] - 16s 163ms/step - loss: 1.0465 - accuracy: 0.6216 - val_loss: 1.1898 - val_accuracy: 0.5696
Epoch 5/20
98/98 [==============================] - 16s 160ms/step - loss: 0.9614 - accuracy: 0.6537 - val_loss: 1.0755 - val_accuracy: 0.6178
Epoch 6/20
98/98 [==============================] - 16s 165ms/step - loss: 0.8872 - accuracy: 0.6815 - val_loss: 1.0424 - val_accuracy: 0.6327
Epoch 7/20
98/98 [==============================] - 15s 156ms/step - loss: 0.8318 - accuracy: 0.7016 - val_loss: 0.9382 - val_accuracy: 0.6660
Epoch 8/20